In [1]:
import pandas as pd
import numpy as np
from statistics import mean
import random

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    KFold,
)
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.linear_model import LinearRegression,SGDRegressor,HuberRegressor,Perceptron,BayesianRidge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.neural_network import MLPRegressor
from sklearn.isotonic import IsotonicRegression

import preprocessing_predictions as prep

Index(['date', 'town_code', 'agas_code', 'accumulated_cases',
       'new_cases_on_date', 'new_recoveries_on_date',
       'new_hospitalized_on_date', 'new_deaths_on_date',
       'new_diagnostic_tests_on_date', 'new_vacc_first_dose_on_date',
       'new_vacc_second_dose_on_date', 'new_vacc_third_dose_on_date'],
      dtype='object')


In [2]:
df = prep.get_prediction_data()
df_measures = prep.get_measures_data()

Index(['date', '_id', 'accumulated_cases'], dtype='object')


In [3]:
countries = df['CountryName'].unique()
print('Countries in df: ',countries)
print('PROBLEMATIC!!')

Countries in df:  ['Israel' 'Netherlands']
PROBLEMATIC!!


In [4]:
folds = KFold(n_splits = 10, shuffle = True, random_state = 32)

In [8]:
x_train = df[["deaths", "cases", "hospitalizations", "vaccinations", "temp"]]
y_train = df["StringencyIndexForDisplay"]

x_train_oxford = df_measures[["deaths", "cases"]]
y_train_oxford = df_measures["StringencyIndexForDisplay"]



In [9]:
# Random Forest
rfr_grid_search = RandomForestRegressor(n_jobs=-1,n_estimators=12,min_samples_split=5,max_depth=21,max_features='log2',criterion='absolute_error')
scores_rfr = cross_val_score(rfr_grid_search, x_train, y_train, cv=folds)
print("Accuracy rfr:", scores_rfr)
print("Mean:", mean(scores_rfr))

Accuracy rfr: [0.80923988 0.66567431 0.47850192 0.56186906 0.42212359 0.87484442
 0.78606878 0.76076474 0.88746887 0.87530541]
Mean: 0.7121860984810039


In [9]:
# Random Forest
# FULL OXFORD SET, ALL COUNTRIES
rfr_grid_search = RandomForestRegressor(n_jobs=-1,n_estimators=12,min_samples_split=5,max_depth=21,max_features='log2',criterion='absolute_error')
scores_rfr = cross_val_score(rfr_grid_search, x_train_oxford, y_train_oxford, cv=folds)
print("Accuracy rfr:", scores_rfr)
print("Mean:", mean(scores_rfr))

In [5]:
# Decision Tree
# FULL OXFORD SET, ALL COUNTRIES
dtr_grid_search = DecisionTreeRegressor(criterion='absolute_error')
scores_dtr = cross_val_score(dtr_grid_search, x_train_oxford, y_train_oxford, cv=folds)
print("Accuracy dtr:", scores_dtr)
print("Mean:", mean(scores_dtr))

Accuracy rfr: [0.42388131 0.42296422 0.42742043 0.41482642 0.43004378]
Mean: 0.42382723184585913


In [ ]:
# FULL OXFORD SET, ALL COUNTRIES
dtr_grid_search = DecisionTreeRegressor(criterion='absolute_error')
scores_dtr = cross_val_score(dtr_grid_search, x_train_oxford, y_train_oxford, cv=folds)
print("Accuracy dtr:", scores_dtr)
print("Mean:", mean(scores_dtr))

array(['Israel', 'Netherlands'], dtype=object)